In [1]:

class SpiderCroppedVideoReader():
    """ This class mimics OpenCV's VideoCapture class, but instead reads
        the *.ufmf files, which contain 200x200 cropped/rotated and monochrome
        spider images.
    """

    def __init__(self, filename, progressCallback=None):
        self.fname = filename
        self.iframe = 0
        self.progressCallback = progressCallback

        # DEBUG
        self.MAX_NUM_FRAMES = 50 * 3600 * 24 * 100

        # Is this a supported input file?
        if not self.fname.endswith('.ufmf'):
            raise Exception('Unsupported input file. Only *.ufmf files allowed.')

        # Open file to determine size
        from motmot.SpiderMovie import SpiderMovie
        mov = SpiderMovie(filename)
        #self.arr = np.memmap(self.fname, dtype=np.uint8, mode='r')
        #N = self.arr.shape[0]
        N = mov.shape[0]*mov.shape[1]*mov.shape[2]
        #del self.arr
        # Ensure this file has the right size
        if N % (1024 * 1024) != 0:
            raise Exception('Image does not have expected size of 1024x1024.')
        # Open again with right shape
        #self.arr = np.memmap(self.fname, dtype=np.uint8, mode='r', shape=(int(N / (1024 * 1024)), 1024, 1024))
        self.matrix_shape = mov.shape
        self.mov = mov

    def get_bbox(self):

        return (0, self.mov.shape[1], 0, self.mov.shape[2])
        
    # Get various metadata
    def get(self, i):
        # Call progress callback
        if self.progressCallback is not None:
            try:
                self.progressCallback(float(i) / self.matrix_shape[0])
            except:
                pass
        # NUM. FRAMES
        if i == 7:
            if self.matrix_shape is not None:
                return min(self.MAX_NUM_FRAMES, self.matrix_shape[0])
            else:
                raise Exception('File closed.')
        # FPS
        elif i == 5:
            return 50
        # HEIGHT
        elif i == 4:
            return 1024
        # WIDTH
        elif i == 3:
            return 1024
        # ERROR
        else:
            raise Exception('Unsupported metadata requested: {}'.format(i))

    def isOpened(self):
        return True

    def read(self):
        import numpy as np
        if self.iframe >= self.get(7):
            return False, np.zeros((1024, 1024), dtype=np.uint8)
        else:
            self.iframe += 1
            return True, self.mov[self.iframe - 1]

In [2]:
direcory = 'Z:/HsinYi/Test Video From Darya/'
project_folder = 'test_clip_dlc_v2/'
basepath = direcory + project_folder
project_name = 'TEST'
scorer = 'PC'

import os, glob


video_list = glob.glob(os.path.join(basepath, 'raw/')+'*.s.ufmf')
video_list


['Z:/HsinYi/Test Video From Darya/test_clip_dlc_v2/raw\\bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf']

In [3]:

date= '2023-11-22'

In [4]:
project = "{pn}-{exp}-{date}".format(pn=project_name, exp=scorer, date=date)

In [3]:
import deeplabcut
deeplabcut.create_new_project(project_name , scorer, video_list, working_directory=basepath, copy_videos= True, videoReader = SpiderCroppedVideoReader)

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)
Created "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc_v2\TEST-PC-2023-11-22\videos"
Created "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc_v2\TEST-PC-2023-11-22\labeled-data"
Created "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc_v2\TEST-PC-2023-11-22\training-datasets"
Created "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc_v2\TEST-PC-2023-11-22\dlc-models"
Copying the videos
\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc_v2\TEST-PC-2023-11-22\videos\bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf
Generated "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc_v2\TEST-PC-2023-11-22\config.yaml"

A new project with name TEST-PC-2023-11-22 is created at \\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc_v2 and a configurable file (config

C:\Users\Gordus_Lab\anaconda3\envs\napari_dlc_trainvideo\Lib\site-packages\deeplabcut\motmot\ufmf\ufmf.py:199: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert key not in value


'\\\\10.99.66.32\\Team Spider\\HsinYi\\Test Video From Darya\\test_clip_dlc_v2\\TEST-PC-2023-11-22\\config.yaml'

In [6]:
config_path = basepath+ '/'+project+'/config.yaml'
cfg = deeplabcut.auxiliaryfunctions.read_config(config_path)
cfg['bodyparts'] =['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25']
deeplabcut.auxiliaryfunctions.write_config(config_path, cfg)

In [7]:
videoname = video_list[0].split('raw\\')[1]
videoname


'bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf'

In [8]:
## Skip extract frames
## Skip label frames

## Generate labeling csv
import os, glob

videoname = video_list[0].split('raw\\')[1]
joint_filename = basepath + '/croprot/' +videoname.replace('.ufmf','_dlc_abs.npy')

from deeplabcut.generate_training_dataset import creating_labeling_csv
creating_labeling_csv.creating_labeling_csv(config_path, basepath, videoname, joint_filename)


CREATING-SOME LABELS FOR THE FRAMES


In [9]:
import deeplabcut
deeplabcut.check_labels(config_path)

C:\Users\Gordus_Lab\anaconda3\envs\napari_dlc_trainvideo\Lib\site-packages\deeplabcut\motmot\ufmf\ufmf.py:199: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert key not in value


Creating images with labels by PC.
This is ufmf video. Randomly selected 25 images to plot.


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:10<00:00,  2.38it/s]

If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


In [5]:

print("CREATING TRAININGSET")
deeplabcut.create_training_dataset(config_path)

CREATING TRAININGSET


  0%|                                                                                        | 0/28914 [00:00<?, ?it/s]C:\Users\Gordus_Lab\anaconda3\envs\napari_dlc_trainvideo\Lib\site-packages\deeplabcut\motmot\ufmf\ufmf.py:199: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert key not in value
100%|████████████████████████████████████████████████████████████████████████████| 28914/28914 [08:57<00:00, 53.76it/s]


The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95,
  1,
  (array([19736, 20841, 22215, ...,  4317,  2514,  4756]),
   array([ 4996,  7622,  6876, ..., 14816,  2448, 23582])))]

In [ ]:
print('Train the network')
deeplabcut.train_network(config_path)

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25]],
 'all_joints_names': ['0',
                      '1',
                      '2',
                      '3',
                      '4',
                      '5',
                      '6',
                      '7',
                      '8',
                      '9',
                      '10',
                      '11',
                      '12',
                      '13',
                      '14',
                   

Train the network
Selecting single-animal trainer
Batch Size is 1


C:\Users\Gordus_Lab\anaconda3\envs\napari_dlc_trainvideo\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Loading ImageNet-pretrained resnet_50


Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\Gordus_Lab\anaconda3\envs\napari_dlc_trainvideo\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\Gordus_Lab\anaconda3\envs\napari_dlc_trainvideo\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Gordus_Lab\anaconda3\envs\napari_dlc_trainvideo\lib\site-packages\deeplabcut\pose_estimation_tensorflow\core\train.py", line 80, in load_and_enqueue
    batch_np = dataset.next_batch()
  File "C:\Users\Gordus_Lab\anaconda3\envs\napari_dlc_trainvideo\lib\site-packages\deeplabcut\pose_estimation_tensorflow\datasets\pose_imgaug.py", line 370, in next_batch
    ) = self.get_batch()
  File "C:\Users\Gordus_Lab\anaconda3\envs\napari_dlc_trainvideo\lib\site-packages\deeplabcut\pose_estimation_tensorflow\datasets\pose_imgaug.py", line 300, in get_batch
    vid_name = self.cfg.project_path + '/videos/' + vid_name
AttributeError: 'dict' object 

Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'Z:\\HsinYi\\Test Video From Darya\\test_clip_dlc_v2\\TEST-PC-2023-11-22\\dlc-models\\iteration-0\\TESTNov22-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'default', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predict': False, 'pairwise_predict': False, 'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [1

In [6]:
import deeplabcut
config_path = basepath+ '/'+project+'/config.yaml'
cfg = deeplabcut.auxiliaryfunctions.read_config(config_path)

deeplabcut.evaluate_network(
    config_path)

Running  DLC_resnet50_TESTNov22shuffle1_50000  with # of training iterations: 50000
This net has already been evaluated!


In [7]:
import deeplabcut
videoname = 'bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf'
deeplabcut.analyze_videos(
    config_path, videos=[basepath+project+'/videos/'+videoname], videoReader = SpiderCroppedVideoReader
)

Using snapshot-50000 for model Z:/HsinYi/Test Video From Darya/test_clip_dlc_v2//TEST-PC-2023-11-22\dlc-models\iteration-0\TESTNov22-trainset95shuffle1


C:\Users\Gordus_Lab\anaconda3\envs\napari_dlc_trainvideo\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
C:\Users\Gordus_Lab\anaconda3\envs\napari_dlc_trainvideo\Lib\site-packages\deeplabcut\motmot\ufmf\ufmf.py:199: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert key not in value


Starting to analyze %  Z:/HsinYi/Test Video From Darya/test_clip_dlc_v2/TEST-PC-2023-11-22/videos/bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf
Loading  Z:/HsinYi/Test Video From Darya/test_clip_dlc_v2/TEST-PC-2023-11-22/videos/bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf
Duration of video [s]:  608.72 , recorded with  50 fps!
Overall # of frames:  30436  found with (before cropping) frame dimensions:  1024 1024
Starting to extract posture


30704it [12:38:15,  1.48s/it]                                                                                          


Saving results in Z:\HsinYi\Test Video From Darya\test_clip_dlc_v2\TEST-PC-2023-11-22\videos...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet50_TESTNov22shuffle1_50000'

In [5]:
import deeplabcut
config_path = basepath+ '/'+project+'/config.yaml'
videoname = 'bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf'
## Note: Since I only train 1000 step, the video should look very bad.
deeplabcut.create_labeled_ufmfvideo(config_path,[basepath+project+'/videos/'+videoname],videotype='ufmf',codec='mp4v')

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)
Starting %  Z:\HsinYi\Test Video From Darya\test_clip_dlc_v2\TEST-PC-2023-11-22\videos ['Z:/HsinYi/Test Video From Darya/test_clip_dlc_v2/TEST-PC-2023-11-22/videos/bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf']
Loading  Z:/HsinYi/Test Video From Darya/test_clip_dlc_v2/TEST-PC-2023-11-22/videos/bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf and data.


C:\Users\Gordus_Lab\anaconda3\envs\napari_dlc_trainvideo\Lib\site-packages\deeplabcut\motmot\ufmf\ufmf.py:199: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert key not in value


False 0 1024 0 1024


  0%|                                                                                         | 0/3000 [00:00<?, ?it/s]C:\Users\Gordus_Lab\anaconda3\envs\napari_dlc_trainvideo\lib\site-packages\deeplabcut\utils\make_labeled_video.py:554: FutureWarning: `draw.circle` is deprecated in favor of `draw.disk`.`draw.circle` will be removed in version 0.19
  rr, cc = circle(yc, xc, dotsize, shape=(ny, nx))
100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:40<00:00, 73.39it/s]


Writing video...
